In [0]:
import torch
from torchvision import datasets, transforms

In [0]:
batch_size = 64

In [3]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu == True:
  print("CUDA is available!  Training on GPU ...")
else:
  print("CUDA is not available.  Training on CPU ...")

CUDA is available!  Training on GPU ...


In [0]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [9]:
dataset = datasets.CIFAR10(
    'data',
    download = True,
    transform = transform
)

Files already downloaded and verified


In [10]:
!ls

data  sample_data


In [11]:
dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [12]:
type(dataset)

torchvision.datasets.cifar.CIFAR10

In [0]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)